In [1]:
# !pip install tensorflow -q
# !pip install librosa -q

In [2]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Audio

import os
import sys
import gc

import librosa
import librosa.display
import json
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [3]:
df = pd.read_csv('file_paths_labels.csv')

# Creating Mel Spectrogram images function

In [5]:
def create_mel_spectrogram(filepath, n_mels=128, n_frames=248):
    data, sr = librosa.load(filepath, sr=None)

    # Calculate the number of frames needed to achieve the desired shape
    hop_length = len(data) // n_frames
    
    # Ensure that the hop_length is at least 1
    hop_length = max(1, hop_length)

    # Compute the Mel spectrogram with the desired parameters
    mel_spectrogram = librosa.feature.melspectrogram(y=data, sr=sr, n_mels=n_mels, hop_length=hop_length)
    
    # Resize the spectrogram to the desired shape using NumPy's slicing
    if mel_spectrogram.shape[1] < n_frames:
        mel_spectrogram = np.pad(mel_spectrogram, ((0, 0), (0, n_frames - mel_spectrogram.shape[1])))
    elif mel_spectrogram.shape[1] > n_frames:
        mel_spectrogram = mel_spectrogram[:, :n_frames]

    return librosa.power_to_db(mel_spectrogram, ref=np.max)

In [6]:
# extracting numpy arrays and concatenating
mel_spec_arrays = []

for filepath in tqdm(df['Filename']):
    #print(filepath)
    mel_spec_arrays.append(create_mel_spectrogram(filepath))
    
concat_array = np.stack(mel_spec_arrays, axis = 0)

100%|████████████████████████████████████████████████████████████████████████████| 10602/10602 [08:37<00:00, 20.48it/s]


In [10]:
# Saving numpy arrays
file_path = "mel_spectrogram_arrs.npy"
np.save(file_path, concat_array)

file_path = 'labels.npy'
np.save(file_path, df['Label'].values)